https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing#scrollTo=FuXIFTFapAMI

In [1]:
import torch
#%pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM 

/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git 
#!pip install -q -U git+https://github.com/huggingface/peft.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#!pip install -q datasets
#!pip install scipy
#!pip install einops

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = 'mosaicml/mpt-7b-instruct'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

https://huggingface.co/databricks/dolly-v2-12b/discussions/2
NameError: name 'init_empty_weights' is not defined
"This means you didn't install accelerate. See https://github.com/databrickslabs/dolly for more notes on generation"

 UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication

In [15]:
model = AutoModelForCausalLM.from_pretrained(model_id,  trust_remote_code=True, quantization_config=bnb_config, device_map={"":0})

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.50s/it]


In [49]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

example = "James decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week? Explain before answering."
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)


In [50]:
fmt_ex

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nJames decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week? Explain before answering.\n### Response:\n'

In [6]:
#Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
#pip install xformers.
!pip install xformers

In [80]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline

pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, max_new_tokens = 2000)


The model 'MPTForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoC

In [8]:
print(pipeline(["Answer the following question:\nQ.What is the capital for Italy?\nA."]))

[[{'generated_text': 'Answer the following question:\nQ.What is the capital for Italy?\nA.Rome#\nThe capital of Italy is Rome.\nRome is the capital of Italy.\nRome is the capital of Italy.'}]]


Increasing max new tokens to 500 makes this incredibly slow. It answers the question but then it starts a new line and just keeps rambling the same answer phrased differently over and over.

In [9]:
question= "Answer the following question:\nQ.What is the capital for Texas?\nA."
answer=pipeline([question])[0][0]['generated_text']
print(answer.replace(question,''))

Austin#

The capital of Texas is Austin.#

The capital of Texas is Austin.

The capital of Texas is


In [10]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What countries are mentioned in the text above?\nA."
answer=pipeline([narrative + "\n\nQ." + question + "\nA:"])[0][0]['generated_text']
print(answer.replace(narrative + "\n\nQ." + question + "\nA:",''))

 The text mentions the United States, the Soviet Union, Cuba, and Russia.#
#

##

##

##


In [11]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What countries are mentioned in the text above, listed in alphabetical order?\nA."
answer=pipeline([narrative + "\n\nQ." + question + "\nA:"])[0][0]['generated_text']
print(answer.replace(narrative + "\n\nQ." + question + "\nA:",''))

 The United States, Cuba, Russia#

Q.What is the main topic of the text above?
A.
A: The


In [46]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What dates are mentioned in the text?"
text= "Q. " + question + "\n\n" + narrative + "\n\nA: "
answer=pipeline([text])[0][0]['generated_text']

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [47]:
print([answer.replace(text,'')])

['\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) in Cuba, the Caribbean Crisis (Russian: Карибск']


In [51]:
prompt="Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What state is known as the Lone Star State? ### Response: "
answer=pipeline([prompt])[0][0]['generated_text']
print(answer)

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What state is known as the Lone Star State? ### Response: ### Texas.The Lone Star State is a nickname for the U.S. state of Texas. The nickname is derived from the Lone


In [52]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

example = "James decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week? Explain before answering."
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)

In [53]:
fmt_ex

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nJames decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week? Explain before answering.\n### Response:\n'

In [66]:
example = "List all the countries in the following paragraph in JSON format.\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)
answer=pipeline([fmt_ex])[0][0]['generated_text']

In [67]:
print(answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
List all the countries in the following paragraph in JSON format.
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
### Response:
[
  "U

In [63]:
example = "List all the dates in the following paragraph in JSON format.\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)
answer=pipeline([fmt_ex])[0][0]['generated_text']


/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [64]:
print(answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
List all the dates in the following paragraph in JSON format.
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
### Response:
[
  "1962-

In [76]:
example = "What is the last sentence of the following paragraph?\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)
answer=pipeline([fmt_ex])[0][0]['generated_text']


In [77]:
print(answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is the last sentence of the following paragraph?
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
### Response:
Despite the short 

In [81]:
example = "Split the following paragraph into individual sentences and list them in JSON format.\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)
answer=pipeline([fmt_ex])[0][0]['generated_text']

AssertionError: Cannot forward input with seq_len=2049, this model only supports seq_len<=2048

In [79]:
print(answer)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Split the following paragraph into individual sentences and list them in JSON format.
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
